In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from collections import Counter
warnings.filterwarnings('ignore')
plt.rcParams.update({'figure.max_open_warning': 0})


In [ ]:
# Load dataset (assumes the file 'Netflix Dataset.csv' is in the working directory)
import os
file_path = 'Netflix Dataset.csv'
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Dataset file not found at {file_path}. Please upload the CSV to the notebook's working directory.")

df = pd.read_csv(file_path)
print('Loaded dataset with shape:', df.shape)
print('\nOriginal columns:')
print(df.columns.tolist())

# Show a quick preview
df.head().copy()

In [ ]:
# Normalize column names to lowercase and strip spaces — keep original mapping for reference
orig_cols = df.columns.tolist()
col_map = {c: c.strip().lower().replace(' ', '_') for c in orig_cols}
df.rename(columns=col_map, inplace=True)
print('Normalized columns:')
print(df.columns.tolist())


In [ ]:
# Map common expected column names (flexible)
expected = {
    'title': ['title'],
    'type': ['type'],
    'director': ['director'],
    'cast': ['cast', 'cast_'],
    'country': ['country', 'countries'],
    'date_added': ['date_added', 'release_date', 'release_date_'],
    'rating': ['rating', 'ratings'],
    'duration': ['duration'],
    'listed_in': ['listed_in', 'genre', 'genres']
}

# Create a reverse lookup to find the actual column name present
found = {}
for key, variants in expected.items():
    for v in variants:
        if v in df.columns:
            found[key] = v
            break

print('Found column mapping:')
for k, v in found.items():
    print(f" - {k}: {v}")

# If date column not found, we'll attempt to infer a date column
if 'date_added' not in found:
    # look for any column name containing 'date'
    for c in df.columns:
        if 'date' in c:
            found['date_added'] = c
            print(f"Inferred date column as: {c}")
            break

# If essential columns are missing, inform user but continue
essential_checks = ['title', 'type']
for ec in essential_checks:
    if ec not in found:
        print(f"Warning: could not find expected column for '{ec}'. Some analyses may be limited.")


In [ ]:
# Create standard column names in dataframe for consistent downstream code
for std_col, actual in found.items():
    df[std_col] = df[actual]

# If some standard columns missing, create them with NaNs so code won't break
for std_col in ['title','type','director','cast','country','date_added','rating','duration','listed_in']:
    if std_col not in df.columns:
        df[std_col] = np.nan

# Quick info
print('Final working columns available:')
print([c for c in ['title','type','director','cast','country','date_added','rating','duration','listed_in'] if c in df.columns])
print('\nNull counts for these columns:')
print(df[['title','type','country','date_added','listed_in']].isnull().sum())


In [ ]:
# Fill textual columns
for col in ['director','cast','country','rating','listed_in','duration']:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

# Normalize country: if multiple countries separated by comma, take the first
if 'country' in df.columns:
    df['country'] = df['country'].astype(str).apply(lambda x: x.split(',')[0].strip() if x.strip() and x.lower()!='unknown' else 'Unknown')

# Parse date column robustly
if df['date_added'].isnull().all():
    print('Date column is entirely empty or not found. Year-based analyses will be skipped.')
else:
    # Try to parse with to_datetime with errors='coerce'
    df['date_added_parsed'] = pd.to_datetime(df['date_added'], errors='coerce')
    # If parsing produced NaT for many rows, try alternative parsing (common formats)
    num_parsed = df['date_added_parsed'].notnull().sum()
    print(f'Parsed {num_parsed} rows into date_added_parsed.')
    # Fill a year_added column
    df['year_added'] = df['date_added_parsed'].dt.year
    df['month_added'] = df['date_added_parsed'].dt.month

# Safe genre parsing function
def extract_genres(cell):
    if pd.isna(cell) or str(cell).strip().lower()=='unknown':
        return []
    parts = [p.strip() for p in str(cell).split(',') if p.strip()]
    return parts

# Build a column of genre lists
df['genre_list'] = df['listed_in'].apply(extract_genres)

# Add a helper column for first_genre (for simple grouping)
df['first_genre'] = df['genre_list'].apply(lambda x: x[0] if x else 'Unknown')

print('\nCleaning complete. Sample:')
df[['title','type','country','date_added','date_added_parsed','year_added','first_genre']].head()

In [ ]:
plt.figure(figsize=(6,4))
try:
    sns.countplot(data=df, x='type', order=df['type'].value_counts().index)
    plt.title('Movies vs TV Shows on Netflix')
    plt.xlabel('Type')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.show()
    display(df['type'].value_counts(dropna=False))
except Exception as e:
    print('Could not plot Movies vs TV Shows distribution:', e)


In [ ]:
if 'year_added' in df.columns and df['year_added'].notnull().any():
    yearly = df['year_added'].value_counts().sort_index()
    plt.figure(figsize=(10,5))
    sns.lineplot(x=yearly.index, y=yearly.values, marker='o')
    plt.title('Content Added Over the Years')
    plt.xlabel('Year')
    plt.ylabel('Number of Titles Added')
    plt.tight_layout()
    plt.show()
    display(yearly)
else:
    print('Year information not available — skipping yearly trend plot.')


In [ ]:
try:
    top_countries = df['country'].value_counts().head(10)
    plt.figure(figsize=(10,5))
    sns.barplot(x=top_countries.values, y=top_countries.index)
    plt.title('Top 10 Countries by Content Count')
    plt.xlabel('Number of Titles')
    plt.ylabel('Country')
    plt.tight_layout()
    plt.show()
    display(top_countries)
except Exception as e:
    print('Could not compute country distribution:', e)


In [ ]:
try:
    genre_counts = Counter()
    for gl in df['genre_list']:
        for g in gl:
            genre_counts[g] += 1
    top_genres = pd.DataFrame(genre_counts.most_common(15), columns=['Genre','Count'])
    plt.figure(figsize=(10,6))
    sns.barplot(data=top_genres, x='Count', y='Genre')
    plt.title('Top Genres')
    plt.tight_layout()
    plt.show()
    display(top_genres)
except Exception as e:
    print('Could not compute genres:', e)


In [ ]:
print('--- Strategic Insights (automatically generated) ---')
# Simple automatic insights based on available data
try:
    total = len(df)
    movies = df[df['type'].str.lower()=='movie'].shape[0] if df['type'].notna().any() else None
    tv = df[df['type'].str.lower()=='tv show'].shape[0] if df['type'].notna().any() else None
    if movies is not None and tv is not None:
        print(f'Out of {total} titles: Movies = {movies} ({movies/total:.1%}), TV Shows = {tv} ({tv/total:.1%})')
except Exception:
    pass

# Top country
try:
    top_country = df['country'].value_counts().idxmax()
    print(f'Top contributing country: {top_country}')
except Exception:
    pass

# Top genre
try:
    top_gen = top_genres.iloc[0]['Genre'] if 'top_genres' in globals() and not top_genres.empty else None
    if top_gen:
        print(f'Most frequent genre: {top_gen}')
except Exception:
    pass

print('\n--- Recommendations (generic) ---')
print('1. If a few countries dominate the catalog, consider investing in regional content for diversity.')
print('2. Identify rising genres by comparing year-over-year counts (if year data is available).')
print('3. Keep a balanced slate of Movies and TV Shows to serve both binge-watchers and movie viewers.')
